In [1]:
# ICV, impacto 

In [1]:
import pandas as pd
from utils import calcular_inventario, coluna_fantasma, criterio_de_corte, alocar

In [2]:
method = 2

In [3]:
inputed = pd.read_excel('entradas.xlsx')
inputed.fillna(0, inplace=True)

In [4]:
ncols = len(inputed.columns)
nprods = len(inputed)
nprocessos = ncols - 3
processos = [f'P{i}' for i in range(1,nprocessos+1)]

In [5]:
dfA = inputed[inputed['Intermediário']==1][['Produto'] + processos].copy()
dfA.set_index('Produto', inplace=True)
dfA

,P1,P2
Produto,,
coque,1.00,-0.4
aço,-0.04,3.0
fofo,0.00,1.0


In [6]:
dfB = inputed[inputed['Intermediário']==0][['Produto'] + processos].copy()
dfB.set_index('Produto', inplace=True)
dfB

,P1,P2
Produto,,
carvão,-1.000,0.0
co2,0.500,2.0
nox,0.002,7.0


In [7]:
dfK = inputed[inputed['Intermediário']==1][['Produto', 'Fluxo de referência']].copy()
dfK.set_index('Produto', inplace=True)
dfK

,Fluxo de referência
Produto,
coque,0.0
aço,2000.0
fofo,0.0


In [8]:
if nprocessos<len(dfA):
    print(f'Matriz A não é quadrada pois existem {len(dfA)} produtos e somente {nprocessos} processos.')

Matriz A não é quadrada pois existem 3 produtos e somente 2 processos.


# Tem um problema de a matriz A não ser inversível. É um problema de alocação, A tem q ser quadrada

### OPÇÃO 1: COMPLETAR COM LINHA DA MATRIZ IDENTIDADE

In [9]:
dfa, dfb, dfk = coluna_fantasma(dfA, dfB, dfK, nprocessos)

In [10]:
inva, bma, M = calcular_inventario(dfa,dfb,dfk)
M

,Emissão
carvão,-268.096515
co2,1474.530831
nox,4692.225201


### OPÇÃO 2: CRITÉRIO DE CORTE (JOGAR UM PRODUTO PARA FLUXO ELEMENTAR)

In [11]:
corte = 'fofo'
dfa, dfb, dfk = criterio_de_corte(dfA, dfB, dfk, corte)

In [12]:
inva, bma, M = calcular_inventario(dfa,dfb,dfk)
M

,Emissão
carvão,-268.096515
co2,1474.530831
nox,4692.225201
fofo,670.241287


### OPÇÃO 3: ALOCAR

In [13]:
processo = 2
elemento1 = 'aço'
elemento2 = 'fofo'
dfa, dfb, dfk = alocar(dfA, dfB, dfK, nprocessos, processo, elemento1, elemento2)

In [14]:
inva, bma, M = calcular_inventario(dfa,dfb,dfk)
M

,Emissão
carvão,-200.803213
co2,1104.417671
nox,3514.457831


## EXERCÍCIO AULA

## Elabore a matriz  Atil e Btil utilizando os seguintes dados:

### o primeiro processo produz 1 kg de coque, consome 0,04 kg de aço, utiliza 1 kg de carvão (da natureza) e emite 0,5 kg de CO2 e 2 g de NOx para o ar,
### o segundo processo produz 3 kg de aço e 1 kg de ferro fundido e, para tanto, consome 0,4 kg de coque, e emite 2 kg de CO2 e 7 kg de NOx para o meio ambiente.

In [15]:
# Matriz A: Fluxos intermediários (tecnológicos)
coque = {'Produto':'Coque', 'Processo 1':1, 'Processo 2':-0.4}
aco = {'Produto':'Aço', 'Processo 1': -0.04, 'Processo 2': 3}
fofo = {'Produto':'Fofo', 'Processo 1': 0, 'Processo 2': 1}
linhas = [coque, aco, fofo]
nprods = len(linhas)

dfA = pd.DataFrame()
for linha in linhas:
    dfA = dfA.append(linha, ignore_index=True)
dfA = dfA.set_index('Produto')
dfA

,Processo 1,Processo 2
Produto,,
Coque,1.00,-0.4
Aço,-0.04,3.0
Fofo,0.00,1.0


In [16]:
# Matriz B: Fluxos elementares
carvao = {'Produto':'Carvão', 'Processo 1': -1, 'Processo 2': 0}
co2 = {'Produto':'CO2', 'Processo 1': 0.5, 'Processo 2': 2}
nox = {'Produto':'NOx', 'Processo 1': 0.002, 'Processo 2': 7}
linhas = [carvao, co2, nox]

dfB = pd.DataFrame()
for linha in linhas:
    dfB = dfB.append(linha, ignore_index=True)
dfB = dfB.set_index('Produto')
dfB

,Processo 1,Processo 2
Produto,,
Carvão,-1.000,0.0
CO2,0.500,2.0
NOx,0.002,7.0


In [17]:
# k é o fluxo de referência
kcoque = {'Produto':'Coque', 'Valor':0}
kaco = {'Produto':'Aço', 'Valor': 1610}
kfofo = {'Produto':'Fofo', 'Valor':0 }
linhas = [kcoque, kaco, kfofo]

dfk = pd.DataFrame()
for linha in linhas:
    dfk = dfk.append(linha, ignore_index=True)
dfk = dfk.set_index('Produto')
dfk

,Valor
Produto,
Coque,0.0
Aço,1610.0
Fofo,0.0


In [18]:
# mudanças climaticas, acidificacao, toxicidade humana
# impactos = {'CO2':[1, 0, 0], }
# a)	o CO2 contribui com fator 1 para mudanças climáticas (kg eq. CO2) e o CH4 com fator 25;
# b)	o SO2 apresenta fator 1 para acidificação (kg eq. SO2) e o NO2, 0,7 e,
# c)	para a toxicidade humana, o MP tem fator 0,2 e o SO2, fator 0,01